# Unpooled model for NBA players points per games prediction

Amos Colombo, Christian Pala

## Libraries:

In [2]:
%matplotlib inline
import numpy as np
import pymc3 as pm
import pandas as pd
import arviz as az
import seaborn as sns
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.stats import halfnorm
sns.set(rc={'figure.figsize':(12,8)})
sns.set_theme()
az.style.use('arviz-darkgrid')
np.random.seed(44)

WARN: Could not locate executable g77
WARN: Could not locate executable f77
WARN: Could not locate executable ifort
WARN: Could not locate executable ifl
WARN: Could not locate executable f90
WARN: Could not locate executable DF
WARN: Could not locate executable efl
WARN: Could not locate executable gfortran
WARN: Could not locate executable f95
WARN: Could not locate executable g95
WARN: Could not locate executable efort
WARN: Could not locate executable efc
WARN: Could not locate executable flang
WARN: don't know how to compile Fortran code on platform 'nt'


WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Preprocessing

In [5]:
# Data
training = pd.read_pickle(Path('..', 'data', 'train.pkl'))
testing = pd.read_pickle(Path('..', 'data', 'test.pkl'))

# remove the last row with the problematic player
training = training.drop(training.index[[-1]])
testing = testing.drop(testing.index[[-1]])

# Features and labels:
x_train = [np.concatenate(training.iloc[i, 2:42].values).reshape(40, 40) for i in range(5)]
y_train = [training.iloc[i, 42:82].values.astype(np.float64) for i in range(5)]
x_test = [testing.iloc[i, 1].reshape(40, 1) for i in range(5)]
y_test = [testing.iloc[i, 2].astype(np.float64) for i in range(5)]

## Modelling:

Unpooled Linear Regression

We treat each player as an individual unit and fit a separate model for each player.

$$y_{i,j} = \alpha_j + \beta_j x_i + \epsilon_i$$

where:

$y_{i,j}$: the player j's observed score for game i.

$\alpha_j$ : the intercept for player j. We standardized the data, so it represents the mean points per game for player j.

$x_i$: the standardized points scored for game i.

$\beta_j$ : the regression coefficients for the j-th feature of the model for player j.

$\epsilon_i$ random variation of the measures.

In this model, we sample 40 different $\beta_j$ values for each player j, with 5 players

In [ ]:
unpooled_models = []
unpooled_traces = []
for i in range(5):
    # sampling variables
    mu_y = y_train[i].mean()
    s_y = y_train[i].std()
    s_x = np.mean(np.std(x_train[i]))

    # model
    with pm.Model() as unpooled_model:
        # priors:
        alpha = pm.Normal('alpha', mu=mu_y, sigma=s_y)
        beta = pm.Normal('beta', mu=0, sigma=s_x, shape=40)
        sigma = pm.HalfNormal('sigma', sigma=s_y)

        # likelihood:
        mu = alpha + pm.math.dot(x_train[i], beta)

        # observed data:
        y_pred = pm.StudentT ('y_pred', nu=4, mu=mu, sd=sigma, observed= y_train[i])
        trace = pm.sample(1000, tune=1000, cores=2, chains=2)

    unpooled_models.append(unpooled_model)
    unpooled_traces.append(trace)

Due to how computationally costly it was to run these models, we moved to a colab notebook.